In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 64.4 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 6.6 MB 58.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install aitextgen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 572 kB 4.9 MB/s 
     |████████████████████████████████| 87 kB 7.9 MB/s 
     |████████████████████████████████| 585 kB 62.9 MB/s 
     |████████████████████████████████| 140 kB 76.3 MB/s 
     |████████████████████████████████| 419 kB 64.3 MB/s 
     |████████████████████████████████| 1.1 MB 56.2 MB/s 
     |████████████████████████████████| 144 kB 75.4 MB/s 
     |████████████████████████████████| 271 kB 72.1 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
  Created wheel for aitextgen: filename=aitextgen-0.5.2-py3-none-any.whl size=575905 sha256=0d95844da2e65b523562879ddd65319859795adbbd2d9fdb7e5b3c8292321620
  Stored in directory: /root/.cache/pip/wheels/83/e2/74/46c887b0989a51a7acee0c09551a3ae9d34b939fb4bea404a0
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=8bde326310992303b518dc2feb03a038e6

In [3]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/SMU_MITB_NLP/NLP project"

Mounted at /content/drive
/content/drive/MyDrive/SMU_MITB_NLP/NLP project


In [5]:
from aitextgen import aitextgen
from aitextgen.TokenDataset import TokenDataset, merge_datasets
from pytorch_lightning.loggers import TensorBoardLogger

In [6]:
import os
import sys
sys.path.append('./lib')
import utilities
import torch
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16860131469303500387
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 13266321408
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3195705674942356954
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [8]:
torch.cuda.is_available()

True

In [14]:
hf_model = "gpt2"
ai = aitextgen(model=hf_model, verbose=True)
ai.to_gpu()
out_dir = "./output/"

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at aitextgen/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
   

In [15]:
full_corpus = utilities.load_corpus("./songs_edit_revised")
train_corpus, val_corpus, _, _ = utilities.split_corpus("./songs_edit_revised")
train_corpus_preprocess = utilities.remove_punct(utilities.remove_annotation(utilities.to_lower(train_corpus)), keep="'")
f = open("./tmp/train_corpus.txt", "w")
f.write(train_corpus_preprocess)
f.close()

In [16]:
traindata = TokenDataset("./tmp/train_corpus.txt", block_size=32)

  0%|          | 0/2112 [00:00<?, ?it/s]

In [17]:
ai.train(
    traindata,
    n_gpu = 1,
    num_steps = 5000,
    generate_every = 100,
    output_dir = out_dir,
    loggers = [TensorBoardLogger(out_dir)],
    freeze_layers = True,
    num_layers_freeze = 10,
    line_by_line = False,
    header = False,
)

pytorch_model.bin already exists in /./output/ and will be overwritten!
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is depre

  0%|          | 0/5000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2282: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  "`trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7."


100 steps reached: generating sample texts.
your dreams began as if something were more than a few years ago
and now we're singapore
and we are singapore

every time we are a part of the world
we are singapore
we are singapore
our future is yours
we are ours
we are yours
we are yours
we have one thing is yours
we are yours
and mine is yours
and ours are yours is yours
we are yours
we are yours
we are yours
we are yours
we are yours and mine
we are yours
and ours
we are yours
we're yours the people
we are yours
we are yours
we are yours

we have yours
we're yours
we're yours
we're yours
we'd be yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we're yours
we've yours
we're yours
we're yours
we're yours
we're
200 steps reached: generating sample texts.
for me
this is my family is my family
a family of friends that are 

Configuration saved in ./output/config.json


...

Configuration saved in ./output/config.json


5,000 steps reached: saving model to /./output/


Model weights saved in ./output/pytorch_model.bin


5,000 steps reached: generating sample texts.

nothing in this world compares
it's our singaporean life
everyone is family friend and neighbour
living in harmony
nothing in this world compares
to this island where it's home
where we love and know
we'll never be alone
nothing in this world compares
it's our singaporean life
nothing in this world compares
to this world
it's our singaporean




nothing in this world

nothing in this world


nothing inthis world





nothing in this

nothing inthis world
singin this world
singapore



nothing inthis
everyoneit's




ait's
one singapore

everywhere we
sing
it's

you we


nothing inthere's


nothing in
one
nothing in
life

nothing inthis is







it's
everybody
nothing in



nothing inwe
nothing in
in this




nothing inwe can
nothing inone
to this
it's
nothing init's
nothing in
nothing in


nothing in




Configuration saved in ./output/config.json
Model weights saved in ./output/pytorch_model.bin


In [13]:
prompts = ["whenever I think back", "we will rise stronger together", "relight our fire, we will find our way",
           "i am tired of being what you want me to be", "feeling so faithless, lost under the surface",
           "and so this I know"]
for prompt in prompts:
    output = ai.generate(
        prompt = prompt,
        n = 1,
        min_len = 100,
        max_len = 100,
        temperature = 0.8,
        do_sample = True,
        use_cache = True,
        early_stopping = False,
        num_beams = 1,
        top_k = 50,
        top_p = 0.75,
        repetition_penalty = 1.2,
        length_penalty = 1.0,
        no_repeat_ngram_size = 0,
        num_beam_groups = 1,
        diversity_penalty = 0.0,
        remove_invalid_values = True,
        return_as_list = True,
        lstrip = False,
        skip_special_tokens = False
    )
    f = open("./output/human_GPT2_full_corpus_" + prompt + ".txt", "w")
    f.write(output[0])
    f.close()          

In [ ]:
f = open("prompt_ref.json", "r")